# Nijmegen Open Data

Spark Notebook to help you analyze data sets provided by Nijmegen's city council.

## Nijmegen data

The city of Nijmegen provides a variety of resources as open data at
[opendata.nijmegen.nl](https://opendata.nijmegen.nl/).

Our goal is to analyze these together - to investigate if we can say anything about the relation between population statistics in areas of the city and the activities that are organized there.

Specifically, we will integrate the following two data sets:
- Streetnames and their quarters: _not available through portal anymore_, use the `wget` command instead
    * `wget https://raw.githubusercontent.com/rubigdata/course/gh-pages/data/BAG_ADRES.csv`
- Public artworks: https://www.nijmegen.nl/kos/opendata/
    * Due to a mistake in the lastest version of this data, the CSV in this link is corrupt. Use the file `kunstopstraat-kunstwerk.csv` in your repository instead.

You need to download the data yourself and copy the files into the container, e.g. using `docker cp`.
The notebook assumes you choose the CSV version of the datasets, and copy them to a directory `/data/bigdata` in the notebook,
that you may still have to create (e.g., `mkdir -p /data/bigdata` inside the container).

This notebook contains code to get you started in the analysis, so you can pick up Spark by example and refresh your SQL.

To fully understand everything, you will need the documentation: http://spark.apache.org/docs/latest/sql-programming-guide.html

Hints of interesting things to discuss in the blog for assignment 3 start with  _**Q:**_.

Next, more preamble, importing some necessary classes and creating a `SparkSession` object.

In [ ]:
// replacement for old version; used to be 
// val sqlContext = new org.apache.spark.sql.SQLContext(sc)

import org.apache.spark.sql.types._ 

val spark = SparkSession
   .builder()
   .appName("A3b-spark-df")
   .getOrCreate()

### Addresses and Quarters

We start with the so-called BAG (_basisadministratie adressen en gebouwen_, i.e., records of addresses and buildings), a standardized resource for which the definition is managed at the national level.

Like most data, the BAG is distributed as CSV, Comma-Separated-Values. Just using `split` may work on very small datasets with regular data, but as soon as quoted values contain commas, we would get in trouble. Spark 2.0 comes with support to parse various data formats, including CSV and JSON, so let us use the standard interfaces.

In [ ]:
//import sparkSession.implicits._

val bagdata = spark.read.format("csv").option("header", true).load("/data/bigdata/BAG_ADRES.csv").cache()

In [ ]:
// Get an impression of the data, easily!
bagdata.printSchema
bagdata.show(2)

#### Real-life data

DataFrames are very handy to get a quick impression of the data, e.g., using `show()`, `sample()` and `describe()`.

In [ ]:
val addrDF = bagdata.select('STRAAT, 'X_COORD, 'Y_COORD, 'WIJK_OMS)

In [ ]:
// Show 5 rows, without truncating values
addrDF.show(5, false)

In [ ]:
// Describe statistics of the value distribution
addrDF.describe().show()

If you think a bit longer about the reported statistics, you realize that not all the data is equally well formatted - the data dump contains address records without x and/or y coordinates; which you can validate as follows:

In [ ]:
addrDF.filter( $"X_COORD".isNull ).count

How to proceed is a decision that the data scientist needs to take - that is, it will be up to your judgement. Can we proceed without these records? In this case, the inconsistent records concern only a very small fraction of the data (45 out of almost 100K records), and we might decide to simply ignore these without valid `(x,y)` coordinates.

In many real life situations, however, we would need to invest more effort and clean the data to overcome these deficiencies. If you look into examples of records without coordinates (_Q: can you write the small numbers of lines of code to inspect these records?_), you find that some of these records have a value "Niet authentiek" in field STATUS, whereas almost all records have "Naamgeving uitgegeven". We lack the domain knowledge to understand the details of this labelling, and may need to dig deeper, or even have a chat with the owners of the data to learn how these values are assigned.

Keep in mind that we only discovered this anomaly in the data when we attempted to define a proper schema, and discovered that the data did not satisfy that schema. Switching from transformations over RDDs to the more structured Data Frame representation can help avoid mistakes in the analysis.

**Q:** _Discuss decisions in data selection and data cleaning that you took to complete the assignment in your blog post._

#### Schema

The Data Frame API supports a more structured interface to data by defining a `case class` to represent the data.

In [ ]:
case class Addr(street:String, quarter:String, x:Float, y:Float)

Project the desired columns onto the fields of the `case class` and convert to a so-called `dataset`.

In [ ]:
val addrDF = bagdata.select($"STRAAT" as "street",
                            $"X_COORD".cast(FloatType) as "x",
                            $"Y_COORD".cast(FloatType) as "y",
                            $"WIJK_OMS" as "quarter").as[Addr].cache()
addrDF.show(2)

Actually, parsing the X and Y coordinates is a little messy: the data contains the X and Y coordinates in the Dutch locale, which means that they are written as 0,5 instead of 0.5.

When parsing fails, we end up with `null` values.

We define a function to convert values where possible.

In [ ]:
// Use Java's NumberFormat to parse values by their locale 
import java.text.NumberFormat
import java.util.Locale
val nf = NumberFormat.getInstance(Locale.forLanguageTag("nl")); // Handle floats written as 0,05 instead of 0.05

def convToFloat(s: String): Option[Float] = {
  try {
    Some(nf.parse(s).floatValue)
  } catch {
    case e: Exception => None
  }
}

So what does this actually do? Well, here is an example of using the new `toFloat` class on _some_ data with one clear exception:

In [ ]:
Seq( "0.045", "0,054", "abc", "45.789,34" ).map( x => convToFloat(x).getOrElse(null))

We have to register this function as a _user-defined function_ so it can be used in Spark SQL.

In [ ]:
// Registering a user-defined function that handles null values as well
val tfloat = udf((f: String) => convToFloat(f).getOrElse(0f))

// Apply the transformation to the x and y columns
val addrDF = bagdata.select($"STRAAT" as "street",
                            tfloat($"X_COORD").cast(FloatType) as "x",
                            tfloat($"Y_COORD").cast(FloatType) as "y",
                            $"WIJK_OMS" as "quarter").as[Addr].cache()
addrDF.show(2)

Let's check that the missing values have been placed on the origin:

In [ ]:
printf(
  "%d points at origin, %d null values",
  addrDF.filter("x = 0 or y = 0").count,
  addrDF.filter('x.isNull or 'y.isNull).count
)

Use `describe` to get an overview of the data:

In [ ]:
addrDF.describe().show() // Note: describe() returns a dataframe, to which we apply the show() command

#### Using Data Frame Operators

In [ ]:
val qc_1 = addrDF.groupBy("quarter").count.cache()

In [ ]:
qc_1.show(5)

What are the top 10 largest quarters of Nijmegen?

In [ ]:
val qc_1_top = qc_1.orderBy(desc("count")).limit(10)
qc_1_top.show()

Wondering what happens inside? The `explain` operator gives you the _physical_ query plan. If you add a boolean, you also see the logical plans, both the initial as parsed, and the plan that is the result of query optimization.

_It is okay if you do not fully understand the full query plan - but do try for a minute or so._

In [ ]:
println("Group by and count:")
println("===================")
qc_1.explain(true)

println("Order descending limit 10:")
println("==========================")
qc_1_top.explain(true)

#### Using SQL

You are not restricted to building queryplans yourself by applying operators to Data Frames; instead, you can also use the SQL interface, and, mix and match SQL querying with follow-up operations using the Data Frame API, or even convert the data back to RDDs and continue to perform analyses directly working with RDDs.

Using SQL is most convenient when queries get larger and more complicated. Consider for example the query for the 10 largest quarters:

In [ ]:
addrDF.createOrReplaceTempView("addresses")

In [ ]:
val qc_2_top = spark.sql("SELECT quarter, count(quarter) AS qc FROM addresses GROUP BY quarter ORDER BY qc DESC LIMIT 10")

In [ ]:
qc_2_top.show

In [ ]:
qc_2_top.explain(true)

### Artworks

Now that we have the address data prepared, move on to look into the data about the artworks.

Load the data, look at the schema and global statistics, and inspect a small sample:

In [ ]:
val kunst = spark.read
    .format("csv")
    .option("header", "true") // Use first line of all files as header
    .option("inferSchema", "true") // Automatically infer data types
    .load("/mnt/bigdata/kunstopstraat-kunstwerk.csv").cache()


kunst: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [naam: string, bouwjaar: string ... 8 more fields]


In [ ]:
kunst.printSchema

root
 |-- naam: string (nullable = true)
 |-- bouwjaar: string (nullable = true)
 |-- kunstenaar: string (nullable = true)
 |-- locatie: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- omschrijving: string (nullable = true)
 |-- eigendom: string (nullable = true)
 |-- bron: string (nullable = true)
 |-- url: string (nullable = true)



In [ ]:
// Select all the public art created before the year 2000
val kunstwerken = kunst.select("naam","locatie","latitude","longitude","bouwjaar","url").where("bouwjaar <= 1999")

kunstwerken: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [naam: string, locatie: string ... 4 more fields]


In [ ]:
kunstwerken.sample(true,0.1).show()

+--------------------+--------------------+----------+----------+--------+--------------------+
|                naam|             locatie|  latitude| longitude|bouwjaar|                 url|
+--------------------+--------------------+----------+----------+--------+--------------------+
|              Reli?f|   Begijnenstraat 29|51.8485121|5.86058232|    1618|                null|
|          Zeemeermin|      Oude Haven 104|51.8495582|5.86167190|    1800|http://www.nijmeg...|
|Poort Protestants...|   Begijnenstraat 29|51.8484990|5.86062360|    1817|http://www.nijmeg...|
|          Sint Jozef|Van Berchenstraat...|51.8443204|5.85847991|    1881|http://www.nijmeg...|
|               Leeuw|    Kronenburgerpark|51.8459542|5.85780483|    1886|                null|
|             Madonna|     Dobbelmannweg 1|51.8303817|5.84731902|    1909|http://www.nijmeg...|
|Johanna de Lestonnac|     Dobbelmannweg 5|51.8293176|5.84705212|    1912|http://www.nijmeg...|
|   Vier gevelbeelden| Berg en Dalseweg 

The coordinates are not correctly detected as floats - again, due to the Locale.
A quick hack (but not robust code) to cast the values to floats:

In [ ]:
// UGLY: this string transform needs to be replaced by proper Locale handling...
val kunstxy = kunstwerken
  .withColumn("latitude", translate(kunstwerken.col("latitude"),",",".").cast("float"))
  .withColumn("longitude", translate(kunstwerken.col("longitude"),",",".").cast("float"))

Let's inspect some of the data, using global information and a sample.

In [ ]:
kunstxy.sample(true,0.05).show()

Artworks created during WWII:

In [ ]:
kunstxy.filter("bouwjaar >= 1940 and bouwjaar <= 1945").show()

In [ ]:
kunstxy.describe().show()

Many items have no URL information, given the difference in counts between `url` and `latitude`.

Let's inspect the problematic tuples (using SQL - _"niet omdat het moet, maar omdat het kan"_):

In [ ]:
kunstxy.createOrReplaceTempView("kunstxy")
// spark.sql("SELECT * FROM kunstxy WHERE locatie IS NULL").show()
spark.sql("SELECT * FROM kunstxy WHERE url IS NULL LIMIT 10").show()

If you would use the data in practice, you can simply ignore the `url` column, or define a new table that excludes the NULL values.

Let us however go back to the source data, in the Dataframe called `kunst`, and look into the dataset in more detail:

In [ ]:
kunst.createOrReplaceTempView("kunst")
kunst.describe().show()

Even the counts of tuples with longitude values does not equal the counts with latitude values... and a mean "bouwjaar" in the future is, well, suspicious!

Let's investigate in more detail, listing missing lat/lon and out-of-range year values in a few SQL queries.

__Q:__ _Try to understand the various data problems we encounter when working on this dataset. Which are fundamental, and which are just an artifact of our processing?_

In [ ]:
spark.sql("select * from kunst where (latitude is not null and longitude is null) or (latitude is null and longitude is not null)").show(15)

In [ ]:
spark.sql("select * from kunst where bouwjaar > 2017").show(10)

In [ ]:
spark.sql("select * from kunst where bouwjaar is null").show(10)

### Art on the map

To continue, let us first create an artworks dataset that is relatively clean, removing tuples that resulted from parsing errors and/or those with missing values.

Can we then find out which quarters have the most artworks?
Perhaps we can even identify the development of the city over time using the dates of these artworks?

To answer these questions, we need to join the dataset with addresses with the one with the artworks.
The best link between the two datasets seems to be the coordinates. Let's see how to exploit the location fields to answer our questions.

In [ ]:
val ks = spark.sql("select * from kunst where (latitude is not null and longitude is not null) and bouwjaar < 9999")

In [ ]:
ks.describe()

In [ ]:
val kos = ks
  .withColumn("latitude", translate(ks.col("latitude"),",",".").cast("float"))
  .withColumn("longitude", translate(ks.col("longitude"),",",".").cast("float"))
  .withColumn("bouwjaar",col("bouwjaar").cast("int"))
  .select("naam","locatie","latitude","longitude","bouwjaar","url")

In [ ]:
kos.createOrReplaceTempView("kos")
kos.printSchema

In [ ]:
kos.show(15, false)

In [ ]:
// Metadata from the catalogue
spark.catalog.listDatabases.show(false)
spark.catalog.listTables.show(false)

#### Coordinates, coordinates

Both datasets include (x,y) locations, but they are in different coordinate systems. BAG uses an official Dutch system known as RD New, from "Rijksdriehoeksco??rdinaten" (~ "national triangle coordinates").

The other dataset uses (lat,lon) coordinates to show artworks on the map, see e.g.:
["de pleinenroute"](http://www.nijmegen.nl/kos/kunstroute.aspx?id=1) (Route of the squares) 

Luckily, we are not the first who need to convert values between coordinate systems - I found the following "easy-to-use" (compared to geo-informatics alternatives) 
Java library:
[Coordinate Transformation Suite (CTS)](https://github.com/orbisgis/cts). 

_(We already loaded this library using the `:dp` directive at the top of the notebook.)_

The following snippet of Java/Scala code sets up the library to transform map coordinates to RD. To use an external library in Spark, all the objects must be serializable; as they are shipped to the worker nodes. Here, this is achieved by using the `@transient` directive to instruct Scala not to include this part of the
object in the serialization, in combination with checking for `null` values when using these variables.

More information, if you want to dig deeper:
* RD New, or "Amersfoort": [Wikipedia entry](https://nl.wikipedia.org/wiki/Rijksdriehoeksco%C3%B6rdinaten)
* The transformation: http://pdok-ngr.readthedocs.io/handleidingen.html#coord-trans
* Serialization and `object` vs. `class`: http://spark.apache.org/docs/latest/programming-guide.html#passing-functions-to-spark
* Role of "annotation" `@transient`: http://fdahms.com/2015/10/14/scala-and-the-transient-lazy-val-pattern/

_Understanding all the details of the `CT` class and its inner workings is not necessary to complete assignment 3._

In [ ]:
object CT extends Serializable {
  
  import org.cts.CRSFactory;
  import org.cts.crs.GeodeticCRS;
  import org.cts.registry.EPSGRegistry;
  import org.cts.op.CoordinateOperationFactory;
  import org.cts.op.CoordinateOperation;

  // global variables to keep state for transformations
  @transient private var xy2latlonOp : CoordinateOperation = null;
  @transient private var latlon2xyOp : CoordinateOperation = null;

  // Create the coordinate transformation functions to convert from RD New to WGS:84 and vice versa
  def initTransforms : (CoordinateOperation, CoordinateOperation) = {
    // Create a new CRSFactory, a necessary element to create a CRS without defining one by one all its components
    val cRSFactory = new CRSFactory();

    // Add the appropriate registry to the CRSFactory's registry manager. Here the EPSG registry is used.
    val registryManager = cRSFactory.getRegistryManager();
    registryManager.addRegistry(new EPSGRegistry());

    // CTS will read the EPSG registry seeking the 4326 code, when it finds it,
    // it will create a CoordinateReferenceSystem using the parameters found in the registry.
    val crs1 : GeodeticCRS = (cRSFactory.getCRS("EPSG:28992")).asInstanceOf[GeodeticCRS];
    val crs2 : GeodeticCRS = (cRSFactory.getCRS("EPSG:4326") ).asInstanceOf[GeodeticCRS];
    
    // Transformation (x,y) -> (lon,lat)
    val xy2latlonOps = CoordinateOperationFactory.createCoordinateOperations(crs1,crs2);
    val xy2latlon = xy2latlonOps.iterator().next(); //get(0);
    
    val latlon2xyOps = CoordinateOperationFactory.createCoordinateOperations(crs2,crs1);
    val latlon2xy = latlon2xyOps.iterator().next(); //get(0);
    
    (xy2latlon, latlon2xy)
  }

  // Encapsulate private transient variable (for serializability of the object)
  def getXYOp : CoordinateOperation = {
    if (xy2latlonOp == null){
      val ts = initTransforms
      xy2latlonOp = ts._1
      latlon2xyOp = ts._2
    }
    xy2latlonOp
  }

  // Encapsulate private transient variable (for serializability of the object)
  def getLatLonOp : CoordinateOperation = {
    if (latlon2xyOp == null){
      val ts = initTransforms
      xy2latlonOp = ts._1
      latlon2xyOp = ts._2
    }
    latlon2xyOp
  }
  
  // Use the library's transformation function to convert the coordinates
  def transformXY(x:Float, y:Float) : (Float, Float) = {   
    // Note: easily confused, (lat,lon) <-> (y,x)
    val lonlat = this.getXYOp.transform(Array(x.toDouble, y.toDouble));
    return ( lonlat(1).toFloat, lonlat(0).toFloat)
  }
  
  // Use the library's transformation function to convert the coordinates
  def transformLatLon(lat:Float, lon:Float) : (Float, Float) = {
    // Note: easily confused, (lat,lon) <-> (y,x)
    val xy = this.getLatLonOp.transform(Array(lon.toDouble, lat.toDouble));
    return ( xy(0).toFloat, xy(1).toFloat)
  }
}

Using the transformation from RD New to WGS:84 (the latitude/longitude pairs used in google maps and open streetmap), we can now plot our BAG data on a map.

Consider the following example, where we take a sample of points from the address data that corresponds to _Toernooiveld_ and put these on the map of Nijmegen, using the widgets provided by spark notebook.

In [ ]:
val txyudf = udf( CT.transformXY _ )


In [ ]:
// Register the transformation function as UDF and apply it to the BAG dataframe
val ta = addrDF.withColumn("latlon", txyudf($"x",$"y"))
ta.show

In [ ]:
val ds = ta.filter('street === "Toernooiveld").select('street, 'x, 'y, 'latlon .getField("_1") as "lat", 'latlon .getField("_2") as "lon")
ds.show

In [ ]:
ds.select('lat,'lon)

In [ ]:
val w = GeoPointsChart(ds.select('lat, 'lon))
w

In [ ]:
// Cell left empty on purpose, to encourage you to plot your own street data on the map!



After all this fun with the open address data, we'd almost forget that we set out to join the two datasets!

Let's switch back to SQL, and use the artworks as a starting point.

__Q:__ _Why would you apply the transformation on the artworks dataset to join the result against the addresses, instead of vice versa?_

In [ ]:
spark.udf.register("transformLatLon", (lat:Float, lon:Float) => CT.transformLatLon(lat,lon))

In [ ]:
val kosxy = spark.sql("select naam, bouwjaar, transformLatLon(latitude, longitude) as XY from kos")
kosxy.createOrReplaceTempView("kosxy")

In [ ]:
val kosquarter = spark.sql(
  "select distinct naam, quarter, min(bouwjaar) as jaar from kosxy, addresses " +
  "where abs(XY._1 - x) < 10.0 and abs(XY._2 - y) < 10.0 " +
  "group by naam, quarter "
).cache()

In [ ]:
kosquarter.show(20,false)

Let us now answer the question we started out with initially: can we infer the growth of the city of Nijmegen through the years from the years in which the artworks were created?

In [ ]:
// A nice example where SQL is a good choice to answer the information need:
kosquarter.createOrReplaceTempView("kosquarter")
spark.sql("select distinct quarter, min(jaar) as jaar from kosquarter group by quarter order by jaar").show(100,false)

A few missing city quarters in this list. Do they really have no artworks, or did we not manage to find their corresponding quarter using the spatial join on coordinates?

Inspecting the artworks not matched up with their quarters is easy, and there are quite a few of those:

In [ ]:
// Artworks not found in the spatial join result:
spark.sql("select naam from kos where naam not in (select naam from kosquarter)").count

__Q:__ _Can you produce the list of quarters that is missing because no artwork has been situated in the quarter?_

__Q:__ _Can you produce a longer list of quarters and their oldest artworks?_

__Q:__ _What are the years associated to artworks not yet matched up with the addresses database? What does this mean for our initial research question?_

In [ ]:
// Cell empty on purpose, try your approach here!



Done too quickly? Haven't had enough of open data yet?

Extend your data analysis of the city with more data, digging into the CBS data for Nijmegen.

A variety of statistics about the population has been made available in this dataset: https://open-data.nijmegen.nl/dataset/statistische-data

    wget http://www.nijmegen.nl/opendata/opendata_stadsgetallen.accdb
    
How to convert this Microsoft Access data to CVS: https://rubigdata.github.io/course/background/access2csv.html